In [1]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [2]:
#!jupyter nbextension list

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Store csv created in part one into a DataFrame
file_to_load = "Resources/fileOne.csv"

# Read File and store into Pandas data frame
city_data = pd.read_csv(file_to_load)

In [4]:
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Arraial do Cabo,-22.9661,-42.0278,89.60,74,20,9.22,BR,1616946456
1,Vestmannaeyjar,63.4427,-20.2734,35.60,64,20,11.50,IS,1616946735
2,Natal,-5.7950,-35.2094,87.80,66,40,5.75,BR,1616946737
3,Namatanai,-3.6667,152.4333,79.03,79,100,8.16,PG,1616946739
4,Kavaratti,10.5669,72.6420,84.20,73,100,7.96,IN,1616946740


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_data["Humidity"]
maxhumidity = humidity.max()

In [7]:
# Heatmap of humidity
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
new_cities = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & (city_data["Cloudiness"] == 0), :]
new_cities = new_cities.dropna(how='any')
new_cities.reset_index(inplace=True)
del new_cities['index']
new_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Barcelos,41.5388,-8.6151,73.99,46,0,6.91,PT,1616946874
1,Ţahţā,26.7693,31.5021,75.20,19,0,20.71,EG,1616946910
2,Cape Town,-33.9258,18.4232,75.99,46,0,11.50,ZA,1616946935
3,Carnarvon,-24.8667,113.6333,78.80,83,0,4.61,AU,1616947040
4,San Juan,-31.5375,-68.5364,74.07,24,0,5.01,AR,1616947053


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
new_cities["Hotel Name"] = ""
new_cities.head()
hotel_df = new_cities[["City", "Country", "Lat", "Lng", "Hotel Name"]]
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Barcelos,PT,41.5388,-8.6151,
1,Ţahţā,EG,26.7693,31.5021,
2,Cape Town,ZA,-33.9258,18.4232,
3,Carnarvon,AU,-24.8667,113.6333,
4,San Juan,AR,-31.5375,-68.5364,
5,Pisco,PE,-13.7000,-76.2167,
6,Córdoba,AR,-31.4135,-64.1811,
7,Lima,PE,-12.0432,-77.0282,
8,Calama,CL,-22.4667,-68.9333,
9,Cascais,PT,38.6979,-9.4215,


In [10]:
# Set parameters to search for a hotel

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
}
# Iterate through 
for index, row in hotel_df.iterrows():
    try:
        # get lat, lng from df
        lat = row['Lat']
        lng = row['Lng']
    
        # Use the search term: "Hotel" and our lat/lng
        params['location'] = f"{lat}, {lng}"

        # make request and print url
        response = requests.get(base_url, params=params).json()
    
        # convert to json
   
    
        # Grab the first hotel from the results and store the name
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"

hotel_df



C:\Users\abm1914\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Country,Lat,Lng,Hotel Name
0,Barcelos,PT,41.5388,-8.6151,Casa da Ana Boutique Guest House
1,Ţahţā,EG,26.7693,31.5021,El Salam Hotel
2,Cape Town,ZA,-33.9258,18.4232,Atlanticview Cape Town Boutique Hotel
3,Carnarvon,AU,-24.8667,113.6333,Carnarvon Sea Change Apartments
4,San Juan,AR,-31.5375,-68.5364,Hotel Viñas del Sol
5,Pisco,PE,-13.7000,-76.2167,PiscoMar Peru
6,Córdoba,AR,-31.4135,-64.1811,Selina Nueva Cordoba
7,Lima,PE,-12.0432,-77.0282,Thiana Hotel
8,Calama,CL,-22.4667,-68.9333,ibis budget Calama
9,Cascais,PT,38.6979,-9.4215,Hotel Albatroz Cascais


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))